In [ ]:
pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.9 MB/s eta 0:00:00


# Library define

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torchvision.models import vit_b_16, ViT_B_16_Weights
from torchvision.transforms import Compose, Resize, CenterCrop, Normalize, ToTensor
import underthesea
import json
import pandas as pd
import re
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Trích xuất embedding

## PhoBERT

In [ ]:
model_name = "vinai/phobert-base-v2"
bert_tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def get_sentences_embeddings(context, bert_tokenizer, bert_model, DEVICE = 'cpu'):
    bert_model.to(DEVICE)
    sentences = underthesea.sent_tokenize(context)
    # Lấy 5 câu đầu
    if len(sentences) > 5:
      sentences = sentences[:5]
    all_embeddings = []
    for sentence in sentences:
        inputs = bert_tokenizer(sentence, return_tensors="pt",
                                padding=True, truncation=True, max_length=256)
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]

        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]
            all_embeddings.append(cls_embeddings.squeeze(0))

    return torch.mean(torch.stack(all_embeddings), dim=0)

## Check working

In [ ]:
t1 = get_sentences_embeddings('Ông Nguyễn_Khắc_Chúc đang làm_việc tại Đại_học Quốc_gia Hà_Nội . Bà Lan , vợ ông Chúc , cũng làm_việc tại đây và làm gì đó . Xin chào', bert_tokenizer, bert_model)

In [ ]:
t1.shape

torch.Size([768])

## ViT

In [ ]:
weights_ViT = ViT_B_16_Weights.DEFAULT
model_ViT = vit_b_16(weights=weights_ViT)

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:06<00:00, 51.4MB/s]


In [ ]:
def get_images_embeddings(images_path, path, weights = weights_ViT, model = model_ViT, DEVICE = 'cpu'):
      model = model.to(DEVICE)
      model.heads = torch.nn.Identity()
      model.eval()
      preprocess = weights.transforms()
      all_embeddings = []
      for img_file in images_path:
          correct_path = os.path.join(path, img_file)
          img = Image.open(correct_path).convert('RGB')
          img_tensor = preprocess(img).unsqueeze(0).to(DEVICE)
          with torch.no_grad():
              embedding = model(img_tensor)
              all_embeddings.append(embedding.squeeze(0))

      return torch.mean(torch.stack(all_embeddings), dim=0)

## Check working

In [ ]:
images_path = ['000000000000000.jpg', '000000000000001.jpg', '000000000000003.jpg']

In [ ]:
get_images_embeddings(images_path).shape

torch.Size([768])

## Get embedding of train, dev, test file for text embeddings

In [ ]:
train_path = '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/wseg_train_news.json'
dev_path = '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/wseg_dev_news.json'
test_path = '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/wseg_test_news.json'

In [ ]:
train_labels = []
dev_labels = []
test_labels = []

In [ ]:
with open(train_path, 'r', encoding='utf-8') as f:
  json_data_train = json.load(f)
for doc_id, doc_content in json_data_train.items():
  train_labels.append(doc_content['section'])

with open(dev_path, 'r', encoding='utf-8') as f:
  json_data_dev = json.load(f)
for doc_id, doc_content in json_data_dev.items():
  dev_labels.append(doc_content['section'])

with open(test_path, 'r', encoding='utf-8') as f:
  json_data_test = json.load(f)
for doc_id, doc_content in json_data_test.items():
  test_labels.append(doc_content['section'])

### Extract embedding for training set

In [ ]:
text_embedding_train = []
for doc_id, doc_content in tqdm(json_data_train.items(), desc = 'Processing training set'):
  context = ' '.join(doc_content['context'])
  bert_embedding = get_sentences_embeddings(context, bert_tokenizer, bert_model, device)
  text_embedding_train.append(bert_embedding)
text_embedding_train = torch.stack(text_embedding_train)

Processing training set: 100%|██████████| 12303/12303 [3:45:02<00:00,  1.10s/it]


In [ ]:
torch.save(text_embedding_train, 'text_embedding_train.pt')

In [ ]:
!cp text_embedding_train.pt '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/'

### Extract embedding for dev set

In [ ]:
with open(dev_path, 'r', encoding='utf-8') as f:
  json_data_dev = json.load(f)

In [ ]:
text_embedding_dev = []
for doc_id, doc_content in tqdm(json_data_dev.items(), desc = 'Processing dev set'):
  context = ' '.join(doc_content['context'])
  bert_embedding = get_sentences_embeddings(context, bert_tokenizer, bert_model, device)
  text_embedding_dev.append(bert_embedding)
text_embedding_dev = torch.stack(text_embedding_dev)

Processing dev set: 100%|██████████| 2675/2675 [48:34<00:00,  1.09s/it]


In [ ]:
torch.save(text_embedding_dev, 'text_embedding_dev.pt')

In [ ]:
!cp text_embedding_dev.pt '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/'

### Extract embedding for test set

In [ ]:
with open(test_path, 'r', encoding='utf-8') as f:
  json_data_test = json.load(f)

In [ ]:
text_embedding_test = []
for doc_id, doc_content in tqdm(json_data_test.items(), desc = 'Processing test set'):
  context = ' '.join(doc_content['context'])
  bert_embedding = get_sentences_embeddings(context, bert_tokenizer, bert_model, device)
  text_embedding_test.append(bert_embedding)
text_embedding_test = torch.stack(text_embedding_test)

Processing test set: 100%|██████████| 2656/2656 [52:00<00:00,  1.18s/it]


In [ ]:
torch.save(text_embedding_test, 'text_embedding_test.pt')

In [ ]:
!cp text_embedding_test.pt '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/'

## Get embedding of train, dev, test file for images embeddings

### Extract embedding for training set

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/images/images_train.zip'
destination_path = '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/images/images_train'


with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(destination_path)

print(f"Files extracted to {destination_path}")

Files extracted to /content/drive/MyDrive/Colab Notebooks/MCAN_Data/images/images_train


In [ ]:
image_embedding_train = []
for doc_id, doc_content in tqdm(json_data_train.items(), desc = 'Processing training set'):
  images_path = []
  for img_data in doc_content['images']:
    img_path = img_data['path']
    images_path.append(img_path)
  image_embedding = get_images_embeddings(images_path = images_path, path = '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/images/images_train/images_train', DEVICE = device)
  image_embedding_train.append(image_embedding)
image_embedding_train = torch.stack(image_embedding_train)

Processing training set:  18%|█▊        | 2164/12303 [09:19<06:06, 27.69it/s]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (101756928 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Processing training set: 100%|██████████| 12303/12303 [18:57<00:00, 10.82it/s]


In [ ]:
torch.save(image_embedding_train, 'image_embedding_train.pt')

In [ ]:
from google.colab import files
files.download('image_embedding_train.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp 'image_embedding_train.pt' -d '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/'

In [ ]:
image_embedding_train.shape

torch.Size([12303, 768])

### Extract embedding for dev set

In [ ]:
image_embedding_dev = []
for doc_id, doc_content in tqdm(json_data_dev.items(), desc = 'Processing dev set'):
  images_path = []
  for img_data in doc_content['images']:
    img_path = img_data['path']
    images_path.append(img_path)
  image_embedding = get_images_embeddings(images_path = images_path, path = '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/images/images_dev', DEVICE = device)
  image_embedding_dev.append(image_embedding)
image_embedding_dev = torch.stack(image_embedding_dev)

Processing dev set: 100%|██████████| 2675/2675 [42:17<00:00,  1.05it/s]


In [ ]:
torch.save(image_embedding_dev, 'image_embedding_dev.pt')

In [ ]:
!cp image_embedding_dev.pt '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/'

### Extracting embedding for test set

In [ ]:
image_embedding_test = []
for doc_id, doc_content in tqdm(json_data_test.items(), desc = 'Processing test set'):
  images_path = []
  for img_data in doc_content['images']:
    img_path = img_data['path']
    images_path.append(img_path)
  image_embedding = get_images_embeddings(images_path = images_path, path = '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/images/images_test', DEVICE = device)
  image_embedding_test.append(image_embedding)
image_embedding_test = torch.stack(image_embedding_test)

Processing test set: 100%|██████████| 2656/2656 [33:07<00:00,  1.34it/s]


In [ ]:
torch.save(image_embedding_test, 'image_embedding_test.pt')

In [ ]:
!cp image_embedding_test.pt '/content/drive/MyDrive/Colab Notebooks/MCAN_Data/'

# Create Dataset:

In [ ]:
class NewspaperDataset:
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, index):
        return {
            'embedding': self.embeddings[index],
            'label': torch.tensor(self.labels[index])
        }

In [ ]:
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
dev_labels = label_encoder.transform(dev_labels)
test_labels = label_encoder.transform(test_labels)

In [ ]:
len(train_labels), len(dev_labels), len(test_labels)

(12303, 2675, 2656)

In [ ]:
label_encoder.classes_

array(['Bất động sản', 'Công nghệ', 'Công đoàn', 'Du lịch', 'Giáo dục',
       'Giải trí - Thư giãn', 'Giới trẻ', 'Khoa học', 'Kinh tế',
       'Pháp luật', 'Sức khoẻ - Tình yêu', 'Thế giới', 'Thời sự',
       'Việc làm', 'Văn hoá - Thế thao', 'Xe', 'Đời sống - Xã hội'],
      dtype='<U19')

In [ ]:
label_encoder.classes_.shape

(17,)

# Model Construction

## Text Model

In [ ]:
class TextModel(nn.Module):
    def __init__(self, embedding_dim=768, num_classes = 17):
        super(TextModel, self).__init__()
        self.embedding_dim = embedding_dim
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(256),
            nn.Linear(256, num_classes)
        )

    def forward(self, embedding):
        return self.classifier(embedding)

## Vision Model

In [ ]:
class VisionModel(nn.Module):
    def __init__(self, embedding_dim=768, num_classes = 17):
        super(VisionModel, self).__init__()
        self.embedding_dim = embedding_dim
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(256),
            nn.Linear(256, num_classes)
        )

    def forward(self, embedding):
        return self.classifier(embedding)

## Multimodal Model

In [ ]:
class MultimodalFusionModule(nn.Module):
    def __init__(
        self,
        text_dim,
        image_dim,
        hidden_dim=512,
        num_classes=17,
        dropout=0.3,
        fusion_type="concat"
    ):
        super().__init__()

        self.fusion_type = fusion_type

        # Projection layers
        self.text_projection = nn.Linear(text_dim, hidden_dim)
        self.image_projection = nn.Linear(image_dim, hidden_dim)

        # Attention fusion layers
        if fusion_type == "attention":
            self.attention = nn.MultiheadAttention(
                embed_dim=hidden_dim,
                num_heads=8,
                dropout=dropout,
                batch_first=True
            )
            self.fusion_dim = hidden_dim
        else:
            self.fusion_dim = hidden_dim * 2 if fusion_type == "concat" else hidden_dim

        self.classifier = nn.Sequential(
            nn.Linear(self.fusion_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, text_features, image_features):

        text_proj = self.text_projection(text_features)
        image_proj = self.image_projection(image_features)

        # Apply fusion strategy
        if self.fusion_type == "concat":
            fused_features = torch.cat([text_proj, image_proj], dim=1)

        elif self.fusion_type == "sum":
            fused_features = text_proj + image_proj

        elif self.fusion_type == "attention":
            text_proj = text_proj.unsqueeze(1)
            image_proj = image_proj.unsqueeze(1)

            fused_features, _ = self.attention(
                text_proj,
                image_proj,
                image_proj
            )
            fused_features = fused_features.squeeze(1)

        fused_features = self.dropout(fused_features)
        logits = self.classifier(fused_features)

        return logits

# Model training and evaluating

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in tqdm(train_loader, desc = 'Training'):
            inputs = batch['embedding']
            labels = batch['label']

            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
    print("Training complete.")

In [ ]:
def evaluate_model(model, val_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc = 'Evaluating'):
            inputs = batch['embedding'].to(device)
            labels = batch['label'].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {accuracy:.2f}%")
    return accuracy, all_preds, all_labels

## Text Classification Training and Evaluating

Get embedding file

In [ ]:
text_embedding_train = torch.load('/content/drive/MyDrive/Colab Notebooks/MCAN_Data/text_embedding_train.pt')
text_embedding_dev = torch.load('/content/drive/MyDrive/Colab Notebooks/MCAN_Data/text_embedding_dev.pt')
text_embedding_test = torch.load('/content/drive/MyDrive/Colab Notebooks/MCAN_Data/text_embedding_test.pt')

<ipython-input-22-4eeca87b37b1>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  text_embedding_train = torch.load('/content/drive/MyDrive/Colab Notebooks/MCAN_Data/text_emb

In [ ]:
# Tạo dataset và dataloader
dataset_train = NewspaperDataset(text_embedding_train, train_labels)
dataset_dev = NewspaperDataset(text_embedding_dev, dev_labels)
dataset_test = NewspaperDataset(text_embedding_test, test_labels)

dataloader_train = DataLoader(dataset_train, batch_size=16, shuffle=True)
dataloader_dev = DataLoader(dataset_dev, batch_size=16, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=16, shuffle=True)

In [ ]:
text_model = TextModel().to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(text_model.parameters(), lr=1e-4)

In [ ]:
train_model(text_model, dataloader_train, loss_fn, optimizer, num_epochs=10)
accuracy, predictions, true_labels = evaluate_model(text_model, dataloader_dev, loss_fn)

Training: 100%|██████████| 769/769 [00:08<00:00, 86.45it/s]


Epoch [1/10], Loss: 1.6777


Training: 100%|██████████| 769/769 [00:05<00:00, 128.78it/s]


Epoch [2/10], Loss: 1.1850


Training: 100%|██████████| 769/769 [00:07<00:00, 98.04it/s] 


...

Epoch [9/10], Loss: 1.0161


Training: 100%|██████████| 769/769 [00:10<00:00, 75.41it/s]


Epoch [10/10], Loss: 1.0129
Training complete.


Evaluating: 100%|██████████| 168/168 [00:00<00:00, 609.47it/s]

Validation Loss: 0.7909, Accuracy: 76.93%


In [ ]:
# Đánh giá trên tập test
test_accuracy, test_preds, test_labels = evaluate_model(text_model, dataloader_test, loss_fn)
print(f"Test Accuracy: {test_accuracy:.2f}%")

Evaluating: 100%|██████████| 166/166 [00:00<00:00, 718.41it/s]

Validation Loss: 0.7773, Accuracy: 76.24%
Test Accuracy: 76.24%


In [ ]:
accuracy = accuracy_score(test_labels, test_preds)
precision = precision_score(test_labels, test_preds, average='weighted')
recall = recall_score(test_labels, test_preds, average='weighted')
f1 = f1_score(test_labels, test_preds, average='weighted')
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.76
Precision: 0.74
Recall: 0.76
F1 Score: 0.74


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# In confusion matrix
cm = confusion_matrix(test_labels, test_preds)
print("Confusion Matrix:")
print(cm)

# In báo cáo chi tiết
print("\nClassification Report:")
print(classification_report(test_labels, test_preds, target_names=label_encoder.classes_))

Confusion Matrix:
[[  0   0   0   0   0   0   0   0   4   0   0   0   0   0   0   0   8]
 [  0  75   0   0   0   0   0   0   9   0   0   8   0   0   4   0   4]
 [  0   0 270   0   0   0   0   0   5   1   0   0   0   0   0   0   4]
 [  0   0   0  40   0   0   0   0   8   0   0   1   0   0   6   0   5]
 [  0   0   0   0   0   1   0   0   1   0   0   1   4   0   1   0   5]
 [  0   0   0   0   0  55   0   0   1   0   3   2   1   0  56   0   2]
 [  0   0   4   0   0   0   0   0   2   0   0   0   0   0   5   1   3]
 [  0   1   0   0   0   0   0   0   3   0   1   0   1   0   1   0   0]
 [  0  14   4   3   0   0   0   1 299   5   1   7   2   0  11   8  28]
 [  0   0   3   0   0   3   0   0   0  75   1   1   2   0   1   0  11]
 [  0   0   0   0   0   3   0   0   1   0  36   1   1   0   7   0  15]
 [  0   1   0   3   0   0   0   0   7   1   0  99   1   0   3   0   5]
 [  0   1   3   2   0   0   0   0  11   8   2  18  48   0  12   0  84]
 [  0   0   2   0   0   0   0   0   1   1   0   0   2   0  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Image Classification Training and Evaluating

In [ ]:
image_embedding_train = torch.load('/content/drive/MyDrive/Colab Notebooks/MCAN_Data/image_embedding_train.pt')
image_embedding_dev = torch.load('/content/drive/MyDrive/Colab Notebooks/MCAN_Data/image_embedding_dev.pt')
image_embedding_test = torch.load('/content/drive/MyDrive/Colab Notebooks/MCAN_Data/image_embedding_test.pt')

<ipython-input-21-d0fb30adb3ae>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  image_embedding_train = torch.load('/content/drive/MyDrive/Colab Notebooks/MCAN_Data/image_e

In [ ]:
# Tạo dataset và dataloader
dataset_train = NewspaperDataset(image_embedding_train, train_labels)
dataset_dev = NewspaperDataset(image_embedding_dev, dev_labels)
dataset_test = NewspaperDataset(image_embedding_test, test_labels)

dataloader_train = DataLoader(dataset_train, batch_size=16, shuffle=True)
dataloader_dev = DataLoader(dataset_dev, batch_size=16, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=16, shuffle=True)

In [ ]:
vision_model = VisionModel().to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vision_model.parameters(), lr=1e-4)

In [ ]:
train_model(vision_model, dataloader_train, loss_fn, optimizer, num_epochs=10)
accuracy, predictions, true_labels = evaluate_model(vision_model, dataloader_dev, loss_fn)

Training: 100%|██████████| 769/769 [00:03<00:00, 201.82it/s]


Epoch [1/10], Loss: 2.1475


Training: 100%|██████████| 769/769 [00:02<00:00, 352.85it/s]


Epoch [2/10], Loss: 1.7115


Training: 100%|██████████| 769/769 [00:02<00:00, 377.18it/s]


...

Epoch [9/10], Loss: 1.2590


Training: 100%|██████████| 769/769 [00:01<00:00, 403.43it/s]


Epoch [10/10], Loss: 1.2264
Training complete.


Evaluating: 100%|██████████| 168/168 [00:00<00:00, 683.14it/s]

Validation Loss: 1.3965, Accuracy: 56.26%


In [ ]:
test_accuracy, test_preds, test_labels = evaluate_model(vision_model, dataloader_test, loss_fn)
print(f"Test Accuracy: {test_accuracy:.2f}%")

Evaluating: 100%|██████████| 166/166 [00:00<00:00, 848.26it/s]


Validation Loss: 1.3490, Accuracy: 57.79%
Test Accuracy: 57.79%


In [ ]:
accuracy = accuracy_score(test_labels, test_preds)
precision = precision_score(test_labels, test_preds, average='weighted')
recall = recall_score(test_labels, test_preds, average='weighted')
f1 = f1_score(test_labels, test_preds, average='weighted')
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.58
Precision: 0.54
Recall: 0.58
F1 Score: 0.55


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
cm = confusion_matrix(test_labels, test_preds)
print("Confusion Matrix:")
print(cm)

print("\nClassification Report:")
print(classification_report(test_labels, test_preds, target_names=label_encoder.classes_))

Confusion Matrix:
[[  1   0   0   0   0   0   0   0   7   0   0   0   0   0   0   0   4]
 [  0  45   2   1   0   3   0   0  23   2   2   5   0   0  12   0   5]
 [  0   0 235   0   0   0   0   0   9   6   1   1   3   0  14   0  11]
 [  0   1   1  26   0   1   0   0   2   1   0   2   0   0  11   1  14]
 [  0   0   3   0   0   0   0   0   4   0   1   0   2   0   3   0   0]
 [  0   1   2   1   0  33   0   0   3   0   2   1   0   0  71   0   6]
 [  0   0   3   1   0   3   0   0   1   0   1   0   0   0   4   0   2]
 [  0   0   1   0   0   0   0   0   4   0   0   0   1   0   0   0   1]
 [  1  16  20   2   0   4   0   0 230   4   2   8  11   0  27   6  52]
 [  0   2   9   0   0   1   0   0  16  27   2   2   6   0  12   1  19]
 [  0   1   2   1   0   4   0   0  10   1  17   1   1   0  13   0  13]
 [  0   4   2   5   0   0   0   0  19   2   0  52   3   0  16   2  15]
 [  0   1  20   2   0   1   0   0  33  12   2  17  18   0  23   0  60]
 [  0   0   2   0   0   0   0   0   4   3   0   1   2   0  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Multimodal Classification Training and Evaluating

### Dataloader for multimodal model

In [ ]:
class NewspaperDataset_Multimodal:
    def __init__(self, text_embeddings, image_embeddings, labels):
        """
        Custom dataset for newspaper classification

        Args:
            text_embeddings: PhoBERT embeddings (num_samples, text_dim)
            image_embeddings: ViT embeddings (num_samples, image_dim)
            labels: Classification labels (num_samples,)
        """
        self.text_embeddings = text_embeddings
        self.image_embeddings = image_embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'text_embedding': self.text_embeddings[idx],
            'image_embedding': self.image_embeddings[idx],
            'label': self.labels[idx]
        }

In [ ]:
dataset_train = NewspaperDataset_Multimodal(text_embedding_train, image_embedding_train, train_labels)
dataset_dev = NewspaperDataset_Multimodal(text_embedding_dev, image_embedding_dev, dev_labels)
dataset_test = NewspaperDataset_Multimodal(text_embedding_test, image_embedding_test, test_labels)

dataloader_train = DataLoader(dataset_train, batch_size=16, shuffle=True)
dataloader_dev = DataLoader(dataset_dev, batch_size=16, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=16, shuffle=True)

In [ ]:
multimodal_model = MultimodalFusionModule(
    text_dim=768,
    image_dim=768,
    hidden_dim=512,
    num_classes=17,
    dropout=0.3,
    fusion_type="concat"
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(multimodal_model.parameters(), lr=1e-4)

### Training

In [ ]:
multimodal_model.train()
for epoch in range(10):
  print(f'Epoch {epoch+1}')
  running_loss = 0.0
  for batch in tqdm(dataloader_train, desc='Training'):
            # Move all batch elements to device
            images_embedding = batch['image_embedding'].to(device)
            text_embedding = batch['text_embedding'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = multimodal_model(text_embedding, images_embedding)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

  print(f"Epoch [{epoch+1}/{10}], Loss: {running_loss/len(dataloader_train):.4f}")
print("Training complete.")

Epoch 1


Training: 100%|██████████| 769/769 [00:03<00:00, 236.35it/s]


Epoch [1/10], Loss: 1.5109
Epoch 2


Training: 100%|██████████| 769/769 [00:01<00:00, 393.49it/s]


Epoch [2/10], Loss: 0.9366
Epoch 3


...

Epoch [8/10], Loss: 0.4786
Epoch 9


Training: 100%|██████████| 769/769 [00:01<00:00, 395.96it/s]


Epoch [9/10], Loss: 0.4390
Epoch 10


Training: 100%|██████████| 769/769 [00:02<00:00, 336.42it/s]

Epoch [10/10], Loss: 0.4121
Training complete.


### Evaluating

In [ ]:
multimodal_model.eval()
correct = 0
total = 0
val_loss = 0.0
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(dataloader_dev, desc='Evaluating'):
        images_embedding = batch['image_embedding'].to(device)
        text_embedding = batch['text_embedding'].to(device)
        labels = batch['label'].to(device)

        outputs = multimodal_model(text_embedding, images_embedding)
        loss = loss_fn(outputs, labels)

        val_loss += loss.item()
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = 100 * correct / total
print(f"Validation Loss: {val_loss/len(dataloader_dev):.4f}, Accuracy: {accuracy:.2f}%")

Evaluating: 100%|██████████| 168/168 [00:00<00:00, 791.65it/s]

Validation Loss: 0.7232, Accuracy: 77.16%


### Testing

In [ ]:
multimodal_model.eval()
correct = 0
total = 0
test_loss = 0.0
all_preds_test = []
all_labels_test = []

with torch.no_grad():
    for batch in tqdm(dataloader_test, desc='Evaluating'):
        images_embedding = batch['image_embedding'].to(device)
        text_embedding = batch['text_embedding'].to(device)
        labels = batch['label'].to(device)

        outputs = multimodal_model(text_embedding, images_embedding)
        loss = loss_fn(outputs, labels)

        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_preds_test.extend(predicted.cpu().numpy())
        all_labels_test.extend(labels.cpu().numpy())

accuracy = 100 * correct / total
print(f"Test Loss: {test_loss/len(dataloader_test):.4f}, Accuracy: {accuracy:.2f}%")

Evaluating: 100%|██████████| 166/166 [00:00<00:00, 1070.05it/s]

Test Loss: 0.6867, Accuracy: 77.79%


In [ ]:
accuracy = accuracy_score(all_labels_test, all_preds_test)
precision = precision_score(all_labels_test, all_preds_test, average='weighted')
recall = recall_score(all_labels_test, all_preds_test, average='weighted')
f1 = f1_score(all_labels_test, all_preds_test, average='weighted')
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.78
Precision: 0.78
Recall: 0.78
F1 Score: 0.77


In [ ]:
cm = confusion_matrix(all_labels_test, all_preds_test)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[  3   0   0   0   0   0   0   0   2   0   0   2   2   0   0   0   3]
 [  0  80   0   0   0   0   0   0   6   1   0   6   1   0   2   1   3]
 [  0   0 271   0   0   0   1   0   3   0   0   0   1   3   0   0   1]
 [  1   0   0  45   0   1   0   0   3   0   0   1   0   0   5   0   4]
 [  0   0   1   0   2   0   2   1   0   0   1   0   4   1   0   0   1]
 [  0   0   0   1   0  92   0   0   2   0   2   0   0   0  19   0   4]
 [  0   0   1   1   0   2   6   0   1   0   1   0   0   0   1   0   2]
 [  0   2   0   0   0   0   0   1   1   0   1   0   2   0   0   0   0]
 [  4  18   6   3   0   0   2   0 279   3   0  13  14   2   8   9  22]
 [  0   0   1   0   1   0   0   0   0  76   1   1   5   1   0   1  10]
 [  0   0   0   0   0   1   1   0   0   0  48   1   1   0   1   0  11]
 [  0   4   0   3   0   0   0   0   6   1   0  95   7   0   1   1   2]
 [  0   1   2   4   5   1   5   0  13   7   2   8  83   4   7   1  46]
 [  0   0   2   0   1   0   0   0   1   1   0   0   3   3  

In [ ]:
print("\nClassification Report:")
print(classification_report(all_labels_test, all_preds_test, target_names=label_encoder.classes_))


Classification Report:
                     precision    recall  f1-score   support

       Bất động sản       0.23      0.25      0.24        12
          Công nghệ       0.74      0.80      0.77       100
          Công đoàn       0.94      0.97      0.95       280
            Du lịch       0.61      0.75      0.67        60
           Giáo dục       0.22      0.15      0.18        13
Giải trí - Thư giãn       0.69      0.77      0.72       120
           Giới trẻ       0.23      0.40      0.29        15
           Khoa học       0.50      0.14      0.22         7
            Kinh tế       0.84      0.73      0.78       383
          Pháp luật       0.82      0.78      0.80        97
Sức khoẻ - Tình yêu       0.66      0.75      0.70        64
           Thế giới       0.70      0.79      0.75       120
            Thời sự       0.57      0.44      0.50       189
           Việc làm       0.20      0.18      0.19        17
 Văn hoá - Thế thao       0.89      0.93      0.91       690